In [21]:
# Sam Brown
# sam_brown@mines.edu
# June 11 2025
# Goal: Use the Day class to parse our data and store high and low tide events for each day, eventually creating a feature that identifies which kind of event it is and if the low tide event was skipped.

import my_lib.funcs
import Cycle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Use 2011 to 2013 first to make sure code works, then expand to larger time scales to see if we can find patterns
events_list2011 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2011_2011Events2stas")
events_list2012 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2012_2012Events2stas")
events_list2013 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2013_2013Events2stas")

# Use preprocessing function to get some of the features
pre_11 = my_lib.funcs.extract_event_features(events_list2011)
pre_12 = my_lib.funcs.extract_event_features(events_list2012)
pre_13 = my_lib.funcs.extract_event_features(events_list2013)

# Merge into one large list of Dataframes
tot_dat = pre_11 + pre_12 + pre_13

# List to store instances of Day class
days = []

In [4]:
tot_dat[0].dtypes

station                  object
pre-slip_area           float64
slip_severity           float64
peak_time               float64
total_delta             float64
start_time       datetime64[ns]
dtype: object

In [23]:
# Use the average coordinates for gz stations to retrieve our tide data
x_cor = -168955.1491394913 
y_cor = -599694.5432784811

tide_df = my_lib.funcs.get_tide_height(1100, x_cor, y_cor, "2011-01-01 00:00:00") # Tide height is in centimeters (1100 days = 3 years)

Elapsed time: 28.060953855514526 seconds


In [24]:
# Only down to minutes, need to double check, some values are getting missed
tide_df['time'] = tide_df['time'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M"))

for df in tot_dat:
    df['start_time'] = df['start_time'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M"))

tot_dat = sorted(tot_dat, key=lambda df: df['start_time'].iloc[0])   

In [73]:
# Want to make a list of cycle instances where it is the 
Events = []
i = 0

while i < len(tot_dat) - 1:
    date_time_now = tot_dat[i].at[0, 'start_time']
    date_time_next = tot_dat[i + 1].at[0, 'start_time']

    # Retrieve Tide data for events
    tide1 = tide_df.loc[tide_df['time'] == date_time_now, 'tide_height'].values[0]
    tide2 = tide_df.loc[tide_df['time'] == date_time_next, 'tide_height'].values[0]

    # Classify as high or low tide events
    tideCur = int(tide1 > 0)
    tideNext = int(tide2 > 0)
    
    #Detecting a cycle with a high tide and low tide event
    if tideCur == 1 and tideNext == 0:
        Events.append(Cycle(date_time_now, date_time_next, h_event=True, l_event=True, full_cyc = True))
        i += 2  # skip the next index because it was just used
    else:
        # 1 1 , 0 1, or 0 0
        if tideCur == 1 and tideNext == 1:
            Events.append(Cycle(date_time_now, h_event= True, l_event=False))
            i += 1
        elif tideCur == 0 and tideNext == 1:
            Events.append(Cycle(date_time_now, h_event= False, l_event=True))
            i += 1
        elif tideCur == 0 and tideNext == 0:
            Events.append(Cycle(date_time_now, h_event= False, l_event=True))
            i +=1

    
    
        
    

    

In [75]:
for event in Events:
    print(event.full_cyc, int(event.h_event), int(event.l_event), event.dateH, event.dateL)

False 1 0 2011-01-01 13:00 None
True 1 1 2011-01-02 13:20 2011-01-02 21:30
True 1 1 2011-01-03 13:30 2011-01-03 22:50
True 1 1 2011-01-04 14:50 2011-01-05 03:00
False 1 0 2011-01-05 17:00 None
True 1 1 2011-01-06 15:30 2011-01-07 02:15
False 1 0 2011-01-07 16:20 None
False 1 0 2011-01-08 15:50 None
True 1 1 2011-01-09 16:20 2011-01-10 06:50
False 1 0 2011-01-10 17:35 None
True 1 1 2011-01-11 08:05 2011-01-11 19:10
True 1 1 2011-01-12 08:45 2011-01-12 21:25
False 1 0 2011-01-13 10:35 None
True 1 1 2011-01-14 09:40 2011-01-14 19:45
True 1 1 2011-01-15 09:55 2011-01-15 19:55
False 1 0 2011-01-16 11:15 None
True 1 1 2011-01-17 12:40 2011-01-17 22:05
False 1 0 2011-01-18 13:18 None
True 1 1 2011-01-19 13:55 2011-01-19 23:05
True 1 1 2011-01-20 14:15 2011-01-21 00:30
False 1 0 2011-01-21 16:55 None
True 1 1 2011-01-22 13:35 2011-01-22 23:10
False 1 0 2011-01-23 17:53 None
True 1 1 2011-01-24 11:35 2011-01-25 01:05
False 1 0 2011-01-25 12:50 None
False 1 0 2011-01-26 10:40 None
True 1 1 2011-

In [79]:
tot_dat[1].head()

,station,pre-slip_area,slip_severity,peak_time,total_delta,start_time
0,la01x,178.028005,0.000001,6945.0,0.444825,2011-01-02 13:20
1,la02x,185.148427,0.000001,6930.0,0.470654,2011-01-02 13:20
2,la03x,177.689123,0.000001,6930.0,0.451244,2011-01-02 13:20
3,la04x,122.731570,0.000001,6885.0,0.504946,2011-01-02 13:20
4,la05x,64.906587,0.000002,6855.0,0.515610,2011-01-02 13:20
